# Import statements

In [1]:
%run "ZeroMQ Python Connector".py
%load_ext autoreload
%autoreload
%matplotlib notebook
import pandas as pd
from functools import partial

# Initialisation and Testing

In [2]:
#Create connector instance, with particular port for PUSH, PULL and SUB
connector = DWX_ZeroMQ_Connector()

[INIT] Ready to send commands to METATRADER (PUSH): 32768
[INIT] Listening for responses from METATRADER (PULL): 32769
[INIT] Listening for market data from METATRADER (SUB): 32770


In [3]:
ID = connector._DWX_MTX_SEND_RATESDATA_REQUEST_(_symbol='GBPUSD', _start=(Timestamp.now()-Timedelta(days=100)).strftime('%Y.%m.%d %H:%M:00'), _timeframe=1440, _end=Timestamp.now().strftime('%Y.%m.%d %H:%M:00'))
response = connector._get_response_(messageID=ID)
print("\n This is the response: ", response)

{'_messageID': 1424825326, '_action': 'DATA', '_data': {'2019.02.18 00:00': [1.29041, 1.29387, 1.28916, 1.29223, 114717, 0, 0], '2019.02.19 00:00': [1.29176, 1.3073, 1.28956, 1.30609, 120573, 0, 0], '2019.02.20 00:00': [1.30602, 1.3109, 1.30117, 1.30488, 126974, 0, 0], '2019.02.21 00:00': [1.30461, 1.30951, 1.3026, 1.30371, 135619, 0, 0], '2019.02.22 00:00': [1.30248, 1.308, 1.29679, 1.3054, 127336, 0, 0], '2019.02.25 00:00': [1.30499, 1.31144, 1.30497, 1.30951, 125833, 0, 0], '2019.02.26 00:00': [1.31167, 1.32879, 1.31141, 1.32491, 160731, 0, 0], '2019.02.27 00:00': [1.32513, 1.335, 1.32331, 1.33078, 152570, 0, 0], '2019.02.28 00:00': [1.33069, 1.33204, 1.32534, 1.32594, 135482, 0, 0], '2019.03.01 00:00': [1.32614, 1.32862, 1.31718, 1.32107, 136280, 0, 0], '2019.03.04 00:00': [1.32397, 1.32542, 1.31663, 1.31778, 141927, 0, 0], '2019.03.05 00:00': [1.31733, 1.31984, 1.30974, 1.31764, 144476, 0, 0], '2019.03.06 00:00': [1.31737, 1.31813, 1.31239, 1.31689, 132293, 0, 0], '2019.03.07 00:0

# Order Testing

In [ ]:
response = Series(connector._get_response_())
responseData = Series(response["_data"])
print(type(responseData[0]))
order = connector._generate_default_order_dict()
print(order)

In [ ]:
order = connector._generate_default_order_dict()
connector._DWX_MTX_SEND_MARKETDATA_REQUEST_(_symbol='GBPUSD', _start=(Timestamp.now()-Timedelta(minutes=1)).strftime('%Y.%m.%d %H:%M:00'),_timeframe=1,_end=Timestamp.now().strftime('%Y.%m.%d %H:%M:00'))
response = Series(connector._get_response_())
responseData = Series(response["_data"])
order['_symbol'] = 'GBPUSD'
order['_price'] = responseData[0]
#connector._DWX_MTX_NEW_TRADE_(order)

In [ ]:
message = connector._DWX_MTX_GET_ALL_OPEN_TRADES_()
print(message)

# Obtaining Market Data Chunks

In [4]:
def obtainMarketDataChunk(symbol,startTime, endTime, timeframe=1, data='DATA', chunkSize = 125):
    
    ## Required Variables 
    outputFrame = pd.DataFrame()
    chunkTime = timeframe * chunkSize 
    pdStartTime = Timestamp.strptime(startTime,'%Y.%m.%d %H:%M:00')
    pdEndTime = Timestamp.strptime(endTime,'%Y.%m.%d %H:%M:00')
    timeDelta = pdEndTime - pdStartTime 
    divMod = divmod(timeDelta,Timedelta(minutes=chunkTime))
    currentStartTime = pdStartTime
    currentEndTime = pdEndTime
    response = None
    
    if data == 'RATESDATA':
        requestDataFunction = partial(connector._DWX_MTX_SEND_RATESDATA_REQUEST_,_symbol=symbol, _timeframe=timeframe)
    elif data == 'DATA':
        requestDataFunction = partial(connector._DWX_MTX_SEND_MARKETDATA_REQUEST_,_symbol=symbol, _timeframe=timeframe)
    
    print(divMod) # For testing
    
    for x in range(divMod[0]):
        print("--------------------------------------------------------------------------------------------------")
        oldResponse = connector._get_response_()
        currentEndTime = pdStartTime + (x+1)*Timedelta(minutes=(chunkTime))
        print(currentStartTime.strftime('%Y.%m.%d %H:%M:00'))
        print(currentEndTime.strftime('%Y.%m.%d %H:%M:00'))
        ID = requestDataFunction(_start=currentStartTime.strftime('%Y.%m.%d %H:%M:00')
                                 ,_end=currentEndTime.strftime('%Y.%m.%d %H:%M:00'))
        
        response = connector._get_response_(messageID = ID, maximumTime=8)
        outputFrame = outputFrame.append(pd.DataFrame.from_dict(response['_data'],orient='index')) # Append to dataframe
        currentStartTime = currentEndTime
        sleep(0.1) # To prevent resource timeout
        print("--------------------------------------------------------------------------------------------------")
        
    
    print("--------------------------------------------------------------------------------------------------")
    ID = requestDataFunction(_start=currentStartTime.strftime('%Y.%m.%d %H:%M:00'),_end=endTime)
    print(currentStartTime.strftime('%Y.%m.%d %H:%M:00'))
    print(endTime)
    response = connector._get_response_(messageID = ID, maximumTime = 8)
    outputFrame = outputFrame.append(pd.DataFrame.from_dict(response['_data'],orient='index'))
    outputFrame = outputFrame.reset_index()
    if data == 'DATA':
        outputFrame.rename(columns={'index':'Date', 0:'Close'}, inplace=True)
    elif data == 'RATESDATA':
        outputFrame.rename(columns={'index':'Date', 0:'Open', 1:'High', 2:'Low', 3:'Close', 4:'TickVol', 5:'Spread', 6:'RealVol'}, inplace=True)
    outputFrame['Date'] = pd.to_datetime(outputFrame['Date'], format='%Y.%m.%d %H:%M')
    outputFrame = outputFrame.set_index('Date')
    print("--------------------------------------------------------------------------------------------------")
    return outputFrame


# Testing Market Data Chunk Retrieval

In [30]:
datachunk = obtainMarketDataChunk('GBPUSD', '2019.05.27 00:00:00', '2019.05.28 00:00:00',timeframe=1, data='DATA')
print(datachunk.to_string())

(11, Timedelta('0 days 01:05:00'))
--------------------------------------------------------------------------------------------------
2019.05.27 00:00:00
2019.05.27 02:05:00
{'_messageID': 465279715, '_action': 'DATA', '_data': {'2019.05.27 00:05': 1.27259, '2019.05.27 00:06': 1.27249, '2019.05.27 00:07': 1.27249, '2019.05.27 00:08': 1.27245, '2019.05.27 00:09': 1.27224, '2019.05.27 00:10': 1.27241, '2019.05.27 00:11': 1.27171, '2019.05.27 00:12': 1.27214, '2019.05.27 00:13': 1.27214, '2019.05.27 00:16': 1.27208, '2019.05.27 00:18': 1.27208, '2019.05.27 00:19': 1.27209, '2019.05.27 00:20': 1.27202, '2019.05.27 00:22': 1.27202, '2019.05.27 00:23': 1.27205, '2019.05.27 00:24': 1.27207, '2019.05.27 00:25': 1.27209, '2019.05.27 00:26': 1.27208, '2019.05.27 00:27': 1.27209, '2019.05.27 00:28': 1.27205, '2019.05.27 00:29': 1.27209, '2019.05.27 00:30': 1.27211, '2019.05.27 00:31': 1.27211, '2019.05.27 00:33': 1.27211, '2019.05.27 00:34': 1.27214, '2019.05.27 00:35': 1.27215, '2019.05.27 00:36

{'_messageID': 156310021, '_action': 'DATA', '_data': {'2019.05.27 06:15': 1.27301, '2019.05.27 06:16': 1.273, '2019.05.27 06:17': 1.27303, '2019.05.27 06:18': 1.27303, '2019.05.27 06:19': 1.27304, '2019.05.27 06:20': 1.27305, '2019.05.27 06:21': 1.27302, '2019.05.27 06:22': 1.27311, '2019.05.27 06:23': 1.273, '2019.05.27 06:24': 1.2731, '2019.05.27 06:25': 1.27302, '2019.05.27 06:26': 1.27312, '2019.05.27 06:27': 1.27303, '2019.05.27 06:28': 1.27314, '2019.05.27 06:29': 1.27312, '2019.05.27 06:30': 1.27313, '2019.05.27 06:31': 1.27316, '2019.05.27 06:32': 1.27319, '2019.05.27 06:33': 1.27315, '2019.05.27 06:34': 1.27321, '2019.05.27 06:35': 1.27323, '2019.05.27 06:36': 1.27323, '2019.05.27 06:37': 1.27323, '2019.05.27 06:38': 1.27341, '2019.05.27 06:39': 1.27333, '2019.05.27 06:40': 1.27331, '2019.05.27 06:41': 1.27339, '2019.05.27 06:42': 1.27343, '2019.05.27 06:43': 1.27341, '2019.05.27 06:44': 1.27351, '2019.05.27 06:45': 1.27339, '2019.05.27 06:46': 1.2735, '2019.05.27 06:47': 1.2

{'_messageID': 1253452190, '_action': 'DATA', '_data': {'2019.05.27 12:30': 1.2706, '2019.05.27 12:31': 1.27072, '2019.05.27 12:32': 1.27076, '2019.05.27 12:33': 1.27084, '2019.05.27 12:34': 1.2708, '2019.05.27 12:35': 1.27087, '2019.05.27 12:36': 1.27083, '2019.05.27 12:37': 1.27081, '2019.05.27 12:38': 1.27059, '2019.05.27 12:39': 1.27061, '2019.05.27 12:40': 1.27049, '2019.05.27 12:41': 1.27048, '2019.05.27 12:42': 1.27039, '2019.05.27 12:43': 1.27048, '2019.05.27 12:44': 1.27048, '2019.05.27 12:45': 1.27048, '2019.05.27 12:46': 1.27048, '2019.05.27 12:47': 1.27037, '2019.05.27 12:48': 1.27018, '2019.05.27 12:49': 1.27007, '2019.05.27 12:50': 1.26997, '2019.05.27 12:51': 1.26995, '2019.05.27 12:52': 1.26967, '2019.05.27 12:53': 1.2698, '2019.05.27 12:54': 1.26989, '2019.05.27 12:55': 1.26978, '2019.05.27 12:56': 1.26947, '2019.05.27 12:57': 1.26927, '2019.05.27 12:58': 1.26929, '2019.05.27 12:59': 1.26928, '2019.05.27 13:00': 1.2691, '2019.05.27 13:01': 1.26914, '2019.05.27 13:02': 

{'_messageID': 76063480, '_action': 'DATA', '_data': {'2019.05.27 18:45': 1.26763, '2019.05.27 18:46': 1.26762, '2019.05.27 18:47': 1.26761, '2019.05.27 18:48': 1.2676, '2019.05.27 18:49': 1.26761, '2019.05.27 18:50': 1.26764, '2019.05.27 18:51': 1.26763, '2019.05.27 18:52': 1.26763, '2019.05.27 18:53': 1.26763, '2019.05.27 18:54': 1.26762, '2019.05.27 18:55': 1.26761, '2019.05.27 18:56': 1.2675, '2019.05.27 18:57': 1.26753, '2019.05.27 18:58': 1.26755, '2019.05.27 18:59': 1.26763, '2019.05.27 19:00': 1.26761, '2019.05.27 19:01': 1.26764, '2019.05.27 19:02': 1.26773, '2019.05.27 19:03': 1.26771, '2019.05.27 19:04': 1.26769, '2019.05.27 19:05': 1.26771, '2019.05.27 19:06': 1.26772, '2019.05.27 19:07': 1.26774, '2019.05.27 19:08': 1.26773, '2019.05.27 19:09': 1.26771, '2019.05.27 19:10': 1.26768, '2019.05.27 19:11': 1.2677, '2019.05.27 19:12': 1.26771, '2019.05.27 19:13': 1.26769, '2019.05.27 19:14': 1.26775, '2019.05.27 19:15': 1.26771, '2019.05.27 19:16': 1.26764, '2019.05.27 19:17': 1

In [33]:
#datachunk.loc['2019-05-27 00:00:00':,'Open':'Close']
datachunk.loc['2019-05-27 00:00:00':,'Close']

Date
2019-05-27 00:05:00    1.27259
2019-05-27 00:06:00    1.27249
2019-05-27 00:07:00    1.27249
2019-05-27 00:08:00    1.27245
2019-05-27 00:09:00    1.27224
2019-05-27 00:10:00    1.27241
2019-05-27 00:11:00    1.27171
2019-05-27 00:12:00    1.27214
2019-05-27 00:13:00    1.27214
2019-05-27 00:16:00    1.27208
2019-05-27 00:18:00    1.27208
2019-05-27 00:19:00    1.27209
2019-05-27 00:20:00    1.27202
2019-05-27 00:22:00    1.27202
2019-05-27 00:23:00    1.27205
2019-05-27 00:24:00    1.27207
2019-05-27 00:25:00    1.27209
2019-05-27 00:26:00    1.27208
2019-05-27 00:27:00    1.27209
2019-05-27 00:28:00    1.27205
2019-05-27 00:29:00    1.27209
2019-05-27 00:30:00    1.27211
2019-05-27 00:31:00    1.27211
2019-05-27 00:33:00    1.27211
2019-05-27 00:34:00    1.27214
2019-05-27 00:35:00    1.27215
2019-05-27 00:36:00    1.27210
2019-05-27 00:37:00    1.27204
2019-05-27 00:38:00    1.27209
2019-05-27 00:39:00    1.27210
                        ...   
2019-05-27 23:30:00    1.26792
201

In [ ]:
datachunk.loc[:,'Open':'Close'].plot()